https://www.metacritic.com/game 에서 크롤링 예정

In [1]:
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

헤더부분 유저에이전트,레퍼러 예제

In [2]:
hur={'User-Agent':'Mozilla','referer':'https://www.metacritic.com/game/playstation-5/elden-ring'}
url='https://www.metacritic.com/game/playstation-5/elden-ring'
req=requests.get(url,headers=hur)
bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8') #beautifulsoup객체선언
q1=bsoup.find("div",{"class":"details main_details"})
temp=bsoup.find("span",{"class":"blurb blurb_expanded"}).text
print(temp)

A New World Created By Hidetaka Miyazaki And George R. R. Martin. ELDEN RING, developed by FromSoftware, Inc. and BANDAI NAMCO Entertainment Inc., is a fantasy action-RPG adventure set within a world created by Hidetaka Miyazaki creator of the influential DARK SOULS video game series; and George R.R. Martin author of The New York Times best-selling fantasy series, A Song of Ice and Fire. Danger and discovery lurk around every corner in FromSoftware's largest game to-date.  Hidetaka Miyazaki, President and Game Director of FromSoftware Inc. Known for directing critically-acclaimed games in beloved franchises including Armored Core, Dark Souls, and Sekiro: Shadows Die Twice.  George R.R. Martin is the #1 New York Times bestselling author of many novels, including the acclaimed series A Song of Ice and Fire - A Game of Thrones, A Clash of Kings, A Storm of Swords, A Feast For Crows, and A Dance with Dragons. As a writer-producer, he has worked on The Twilight Zone, Beauty and the Beast, a

이전까지 크롤링했던 진도부터 하기

정해진 플랫폼에 따라 크롤링하는 함수

In [21]:
def crawl_page(url):
    ret={}
    chk=True
    print(url)
    cnt=0
    while(chk):
        req=requests.get(url,headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36','referer':url}) 
        if(req==None): 
            print("reqNone")
            continue
        bsoup=BeautifulSoup(req.text,'lxml')
        time.sleep(1.5)#크롤링 속도 제한
        if(bsoup==None):
            print("bsoupNone")
            continue
        bmetascore=bsoup.find("span",{"itemprop":"ratingValue"})
        if(bmetascore==None):
            time.sleep(4)
            print("bemtaNone")
            cnt+=1;
            if(cnt>=5):
                cnt=0;
                return False;
            continue
        bsummary=bsoup.find("span",class_="blurb blurb_expanded")
        if(bsummary==None):
            print("bsumm")
            bsummary="None"
        else:
            if(bsummary.string==None):
                bsummary=bsoup.find("span",class_="blurb blurb_collapsed")
                if(bsummary.string!=None):
                    bsummary=bsummary.string
                if(bsummary.string!=None):
                    bsummary=bsummary.string
            else:
                bsummary=bsummary.string
        btitle=bsoup.find("div",{"class":"product_title"}).find("h1")
        if(btitle==None): 
            print("btitleNone")
            continue
        buserscore=bsoup.find("div",class_=re.compile("metascore_w user large game"))
        if(buserscore==None): 
            print("buserNone")
            continue
        bpub=bsoup.find("li",class_="summary_detail publisher").find("a")
        if(bpub==None):
            print("bpubNone")
            continue
        bdev=(bsoup.find("li",class_="summary_detail developer"))
        if(bdev==None):
            bdev="None"
            print("bdevNone")
        else:
            bdev=bdev.find("a")
            bdev=bdev.string
        bgenres=(bsoup.find("li",class_="summary_detail product_genre"))
        if(bgenres==None):
            print("bgenreNone")
            continue
        bgenres=bgenres.find_all("span",class_="data")
        sgenres=""
        for x in range(len(bgenres)):
            sgenres=sgenres+str(bgenres[x].string)+"|"
        bgrating=bsoup.find("li",class_="summary_detail product_rating")
        if(bgrating==None):
            print("bgratingNone")
            bgrating="None"
        else:
            bgrating=bgrating.find("span",class_="data").string
        brelease=bsoup.find("li",class_="summary_detail release_data")
        brelease=brelease.find("span",class_="data")
        print(str(btitle.string)," ",str(brelease.string)," ",str(bmetascore.string)," ",str(buserscore.string)," ",str(bpub.string)[57:-53]," ",str(bdev.string)," ",sgenres," ",bgrating)
        ret={"title":str(btitle.string),"metascore":str(bmetascore.string),"userscore":str(buserscore.string),"publisher":str(bpub.string)[57:-53],"developer":str(bdev),"genres":sgenres,"rating":str(bgrating),"summary":str(bsummary),"release_date":str(brelease.string)}
        chk=False
    return ret
    
def get_page(url):
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url})  
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    bsoup2=bsoup.find_all("td",{"class":"clamp-summary-wrap"})
    lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    for x in bsoup2:
        y=x.find("a")["href"]
        y=y[:-15]
        h=crawl_page("https://www.metacritic.com/"+y)
        if(h==False):
            continue
        df_dic=pd.DataFrame([h])
        lpd = pd.concat([lpd, df_dic], ignore_index=True)
    return lpd
        
def crawl_direct(platform,df):
    url="https://www.metacritic.com/browse/games/release-date/available/"+platform+"/metascore?page="
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url+'0'})
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    lpage_soup=bsoup.find("li",{"class":"last_page"})
    lpage2_soup=lpage_soup.find("a",{"class":"page_num"})
    lpagenum=lpage2_soup.text
    try:
        lpd=pd.read_csv(platform+'.csv')
    except:
        lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    f=open(platform+"_proc.txt","r")
    proc=int(f.readline())
    f.close()
    for x in range(proc,int(lpagenum)):  
        print(url+str(x))
        page=get_page(url+str(x))
        lpd=pd.concat([page,lpd],ignore_index=True)
        lpd.to_csv(platform+'.csv',sep=',',na_rep='NaN',float_format = '%.2f',index = False)
        f=open(platform+"_proc.txt","w")
        f.write(str(x+1))
        f.close()
        print(len(lpd))
crawl_direct('ps4',1)

https://www.metacritic.com/browse/games/release-date/available/ps4/metascore?page=6
https://www.metacritic.com//game/playstation-4/kings-quest-chapter-3-once-upon-a-climb
King's Quest Chapter 3: Once Upon a Climb   Apr 26, 2016   79   7.1   Sierra Games   The Odd Gentlemen   Adventure|General|   E10+
https://www.metacritic.com//game/playstation-4/zen-pinball-2-bethesda-pinball
bgratingNone
Zen Pinball 2: Bethesda Pinball   Dec  6, 2016   79   tbd   Zen Studios   Zen Studios   Action|Pinball|   None
https://www.metacritic.com//game/playstation-4/nier-automata---3c3c1d119440927
NieR: Automata - 3C3C1D119440927   May  2, 2017   79   7.5   Square Enix   PlatinumGames   Role-Playing|Action RPG|   M
https://www.metacritic.com//game/playstation-4/railway-empire
Railway Empire   Jan 30, 2018   79   7.1   Kalypso   Gaming Minds Studios   Simulation|Strategy|Management|Vehicle|Train|Business / Tycoon|   E
https://www.metacritic.com//game/playstation-4/bug-fables-the-everlasting-sapling
Bug Fable

bgratingNone
ZEN Pinball 2: Balls of Glory   Oct 20, 2015   78   6.2   Zen Studios   Zen Studios   Action|Pinball|   None
https://www.metacritic.com//game/playstation-4/olija
Olija   Jan 28, 2021   78   tbd   Devolver Digital   Skeleton Crew Studio   Action|Action Adventure|Platformer|Open-World|2D|   T
https://www.metacritic.com//game/playstation-4/detroit-become-human
Detroit: Become Human   May 25, 2018   78   8.7   SCEA   Quantic Dream   Action Adventure|Open-World|Adventure|3D|Third-Person|   M
https://www.metacritic.com//game/playstation-4/everspace
EVERSPACE   May 22, 2018   78   5.9   ROCKFISH Games   ROCKFISH Games   Simulation|Action Adventure|Space|Open-World|Combat|   T
https://www.metacritic.com//game/playstation-4/cyber-shadow
Cyber Shadow   Jan 26, 2021   78   7.1   Yacht Club Games   Mechanical Head Studios   Action|Platformer|2D|   E10+
https://www.metacritic.com//game/playstation-4/ai-the-somnium-files
AI: The Somnium Files   Sep 17, 2019   78   8.8   Spike Chunsoft  

Dragon's Dogma: Dark Arisen   Oct  3, 2017   78   8.2   Capcom   Capcom   Role-Playing|Action RPG|   M
https://www.metacritic.com//game/playstation-4/dont-starve-console-edition
Don't Starve: Console Edition   Jan  7, 2014   78   7.1   Klei Entertainment   Klei Entertainment   Action Adventure|Fantasy|General|Fantasy|Sandbox|   T
https://www.metacritic.com//game/playstation-4/atelier-lulua-the-scion-of-arland
Atelier Lulua: The Scion of Arland   May 21, 2019   78   6.6   Koei Tecmo Games   Koei Tecmo Games   Role-Playing|Japanese-Style|   T
https://www.metacritic.com//game/playstation-4/super-cloudbuilt
Super Cloudbuilt   Jul 25, 2017   78   6.7   Double Eleven   Coilworks, Double Eleven   Action Adventure|General|   T
https://www.metacritic.com//game/playstation-4/fortnite
Fortnite   Jul 21, 2017   78   3.9   Epic Games   Epic Games   Action|Shooter|Third-Person|Tactical|   T
https://www.metacritic.com//game/playstation-4/heavy-rain-beyond-two-souls-collection
Heavy Rain & Beyond: Two

KeyboardInterrupt: 